## Datasets

Three kinds of datasets have been use for this project:

### 1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.
A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at: <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>

### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at: <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>

### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at: <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago crime data</a>



The assessment was performed to understand the link (if any) between poverty and crime rate in certain communities in Chicago, US.

Execution of cell below avoids prettytable default error.


In [2]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00


To analyze the data using SQL, it first needs to be loaded into SQLite DB.
Three tables in as under:

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**


In [3]:
import csv, sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [4]:
!pip install pandas
!pip install ipython-sql

In [5]:
# Load the SQL magic module
%load_ext sql

Used these dataframes to load data on to the database `FinalDB.db` as required tables.


In [8]:
import pandas
df1 = pandas.read_csv("/content/ChicagoCensusData.csv")

In [9]:
df1.to_sql("CENSUS_DATA", con, if_exists='replace', index=False, method="multi")

78

In [10]:
df2 = pandas.read_csv("/content/ChicagoPublicSchools.csv")

In [11]:
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False, method="multi")

566

In [12]:
df3 = pandas.read_csv("/content/ChicagoCrimeData.csv")

In [13]:
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

533

In [14]:
# Establish a connection between SQL magic module and the database FinalDB.db
%sql sqlite:///FinalDB.db

In [15]:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_CRIME_DATA');

 * sqlite:///FinalDB.db
Done.


count(name)
21


## Assessments performed

Total number of crimes recorded in the crime table.


In [16]:
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_CRIME_DATA');

 * sqlite:///FinalDB.db
Done.


name,type,length(type)
ID,INTEGER,7
CASE_NUMBER,TEXT,4
DATE,TEXT,4
BLOCK,TEXT,4
IUCR,TEXT,4
PRIMARY_TYPE,TEXT,4
DESCRIPTION,TEXT,4
LOCATION_DESCRIPTION,TEXT,4
ARREST,INTEGER,7
DOMESTIC,INTEGER,7


In [20]:
%sql SELECT COUNT("CASE_NUMBER") FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


"COUNT(""CASE_NUMBER"")"
533


Community area names and numbers with per capita income less than 11000 to access vulnerable regions.


In [21]:
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CENSUS_DATA');

 * sqlite:///FinalDB.db
Done.


name,type,length(type)
COMMUNITY_AREA_NUMBER,REAL,4
COMMUNITY_AREA_NAME,TEXT,4
PERCENT_OF_HOUSING_CROWDED,REAL,4
PERCENT_HOUSEHOLDS_BELOW_POVERTY,REAL,4
PERCENT_AGED_16__UNEMPLOYED,REAL,4
PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,REAL,4
PERCENT_AGED_UNDER_18_OR_OVER_64,REAL,4
PER_CAPITA_INCOME,INTEGER,7
HARDSHIP_INDEX,REAL,4


In [22]:
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME< 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


All case numbers for crimes involving minors to analyse the crime rate against minors.


In [23]:
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,2333,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,413,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


All kidnapping crimes involving a child.


In [24]:
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE LIKE 'KIDNAPPING'  ;

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


Distinct kind of crimes that were recorded at schools.


In [25]:
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE  LOCATION_DESCRIPTION LIKE '%SCHOOL%' ;

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
4006321,HL353697,2005-05-04,077XX S BURNHAM AVE,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS",0,0,421,4,7.0,43.0,08B,1196066.0,1854156.0,2005,41.75469107,-87.55703869,"(41.754691074, -87.557038686)"
4430638,HL725506,2005-11-09,048XX N FRANCISCO AVE,484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",1,0,2031,20,40.0,4.0,08B,1156348.0,1932071.0,2005,41.96938944,-87.70048881,"(41.96938944, -87.700488807)"
6644618,HP716225,2008-12-04,030XX S DR MARTIN LUTHER KING JR DR,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",0,0,2112,1,2.0,35.0,08B,1179306.0,1885032.0,2008,41.83981621,-87.61751617,"(41.839816207, -87.617516172)"
2341955,HH639427,2002-09-10,005XX N WALLER AVE,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",0,0,1512,15,29.0,25.0,08B,1138238.0,1903181.0,2002,41.89045993,-87.76778089,"(41.890459933, -87.767780886)"
11110571,JA460432,2017-10-05,076XX S HOMAN AVE,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS",0,0,835,8,18.0,70.0,08B,1155024.0,1853614.0,2017,41.75412154,-87.70746025,"(41.754121535, -87.707460248)"
7399281,HS200939,2010-03-10,053XX W CONGRESS PKWY,1320,CRIMINAL DAMAGE,TO VEHICLE,"SCHOOL, PUBLIC, GROUNDS",0,0,1522,15,29.0,25.0,14,1140822.0,1897164.0,2010,41.8739014,-87.7584391,"(41.873901397, -87.758439102)"
3530721,HK577020,2004-08-23,016XX W JONQUIL TER,2024,NARCOTICS,POSS: HEROIN(WHITE),"SCHOOL, PUBLIC, GROUNDS",1,0,2422,24,49.0,1.0,18,1163893.0,1951001.0,2004,42.0211776,-87.67220843,"(42.021177601, -87.67220843)"
7502426,HS305355,2010-05-13,035XX S WASHTENAW AVE,1821,NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING",1,0,913,9,12.0,58.0,18,1158944.0,1880896.0,2010,41.82890791,-87.69234919,"(41.828907913, -87.692349187)"
8082600,HT315369,2011-05-26,032XX W ADAMS ST,545,ASSAULT,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, GROUNDS",0,0,1124,11,28.0,27.0,08A,1154752.0,1898890.0,2011,41.87837031,-87.70724814,"(41.878370307, -87.707248137)"
7174283,HR585012,2009-10-13,043XX W 79TH ST,1330,CRIMINAL TRESPASS,TO LAND,"SCHOOL, PUBLIC, GROUNDS",1,0,834,8,13.0,70.0,26,1148764.0,1851854.0,2009,41.74941446,-87.7304466,"(41.749414464, -87.730446597)"


Type of schools along with the average safety score against crimes.


In [26]:
%sql SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE)  FROM CHICAGO_PUBLIC_SCHOOLS GROUP  BY "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


5 community areas with highest % of households below poverty line to highlight the most vulnerable communities.


In [18]:
%sql SELECT *  FROM CENSUS_DATA  LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [19]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER, HARDSHIP_INDEX  FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,HARDSHIP_INDEX
Edison Park,9.0,8.0
Mount Greenwood,74.0,16.0
Beverly,72.0,12.0
Norwood Park,10.0,21.0
North Center,5.0,6.0


Community area most prone to crime

In [28]:
%sql SELECT * FROM CHICAGO_CRIME_DATA LIMIT 1;

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)"


In [27]:
%sql SELECT COUNT(CASE_NUMBER) AS FREQUENCY, COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY FREQUENCY DESC ;

 * sqlite:///FinalDB.db
Done.


FREQUENCY,COMMUNITY_AREA_NUMBER
43,25.0
43,None
22,23.0
21,68.0
16,29.0
16,28.0
15,8.0
14,71.0
13,24.0
12,67.0


In [29]:
%sql SELECT community_area_name  FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = 25;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


The name of the community area with highest hardship index (used sub-query).
Same as the assessment before only the method is varying.

In [30]:
%%sql
select hardship_index, community_area_name  from CENSUS_DATA
where HARDSHIP_INDEX  IN (select max(hardship_index) FROM CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


HARDSHIP_INDEX,COMMUNITY_AREA_NAME
98.0,Riverdale


Determine the Community Area Name with most number of crimes using sub-query.


In [31]:
%%sql
SELECT CA.COMMUNITY_AREA_NAME
FROM CENSUS_DATA CA
WHERE CA.COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
